Stanislas Deneuville - Emmanuel Ferrandi - Pol Grisart - Marine Médard
# Project of data science :  Face recognition in a video and counting
12/11/2018

## Part I : Cleaning

__face_detection__ : dectect the faces in a photograph.


In : 
* image : name of the file containing the image in the format jpeg for example

Out : 
* face_locations : list of the location of all faces in the picture in parameter


In [ ]:
def face_detection(image) :
    face_locations = []
    return face_locations

__image_generation__ : generate a black image with white circle whoes location in gave in parameter


In : 
* circle_location : list containing the locations of the white circle that will be add to the picture

Out : 
* image : list of list  representing a black picture with white circles


In [4]:
def image_generation(circle_locations) :
    new_image = [[]]
    return new_image

__resize_image__ : resize the image gave in parameter in a 50x50 pixels image 


In : 
* image : list of list  representing an image in white and black

Out : 
* face_locations : list of list representing the same image than in parameter but with the dimensions 50x50


In [2]:
def resize_imgae(image, nb_circles) :
    final_image = [[]]
    return final_image

__save_image__ : save the image in parameter in the good repository according to nb_circles


In : 
* image : image that we want to save
* nb_circles : int, the number of circles in the image

Out : 
* int : 0 if everything is ok


In [3]:
def save_image(image, nb_circles) :
    return 0

__cleaning__ : clean an image so that we can count faces in it thanks to an ANN

In : 
* image : name of the file containing the image in the format jpeg for example



In [ ]:
def cleaning(image) :
    face_locations = face_detection(image)
    
    nb_circles = len(face_locations)
    
    new_image = image_generation(face_locations)
    
    final_image = resize_image(newImage)
    
    save_image(final_image, nb_circles)